In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('creditcard.csv')
df[14:20]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
14,12.0,-2.791855,-0.327771,1.641750,1.767473,-0.136588,0.807596,-0.422911,-1.907107,0.755713,1.151087,0.844555,0.792944,0.370448,-0.734975,0.406796,-0.303058,-0.155869,0.778265,2.221868,-1.582122,1.151663,0.222182,1.020586,0.028317,-0.232746,-0.235557,-0.164778,-0.030154,58.80,0.0
15,12.0,-0.752417,0.345485,2.057323,-1.468643,-1.158394,-0.077850,-0.608581,0.003603,-0.436167,0.747731,-0.793981,-0.770407,1.047627,-1.066604,1.106953,1.660114,-0.279265,-0.419994,0.432535,0.263451,0.499625,1.353650,-0.256573,-0.065084,-0.039124,-0.087086,-0.180998,0.129394,15.99,0.0
16,12.0,1.103215,-0.040296,1.267332,1.289091,-0.735997,0.288069,-0.586057,0.189380,0.782333,-0.267975,-0.450311,0.936708,0.708380,-0.468647,0.354574,-0.246635,-0.009212,-0.595912,-0.575682,-0.113910,-0.024612,0.196002,0.013802,0.103758,0.364298,-0.382261,0.092809,0.037051,12.99,0.0
17,13.0,-0.436905,0.918966,0.924591,-0.727219,0.915679,-0.127867,0.707642,0.087962,-0.665271,-0.737980,0.324098,0.277192,0.252624,-0.291896,-0.184520,1.143174,-0.928709,0.680470,0.025436,-0.047021,-0.194796,-0.672638,-0.156858,-0.888386,-0.342413,-0.049027,0.079692,0.131024,0.89,0.0
18,14.0,-5.401258,-5.450148,1.186305,1.736239,3.049106,-1.763406,-1.559738,0.160842,1.233090,0.345173,0.917230,0.970117,-0.266568,-0.479130,-0.526609,0.472004,-0.725481,0.075081,-0.406867,-2.196848,-0.503600,0.984460,2.458589,0.042119,-0.481631,-0.621272,0.392053,0.949594,46.80,0.0
19,15.0,1.492936,-1.029346,0.454795,-1.438026,-1.555434,-0.720961,-1.080664,-0.053127,-1.978682,1.638076,1.077542,-0.632047,-0.416957,0.052011,-0.042979,-0.166432,0.304241,0.554432,0.054230,-0.387910,-0.177650,-0.175074,0.040002,0.295814,0.332931,-0.220385,0.022298,0.007602,5.00,0.0


In [0]:
df.fillna(0,inplace=True)
print(df.isnull().sum())

In [0]:
print(df.describe())
print(df.columns)

                Time             V1  ...         Amount          Class
count  234628.000000  234628.000000  ...  234628.000000  234628.000000
mean    80997.737031      -0.059631  ...      91.116960       0.001807
std     40546.212756       1.957887  ...     252.546629       0.042472
min         0.000000     -56.407510  ...       0.000000       0.000000
25%     48489.750000      -0.945713  ...       6.000000       0.000000
50%     74631.000000      -0.023583  ...      23.450000       0.000000
75%    122574.000000       1.248698  ...      79.990000       0.000000
max    148051.000000       2.454930  ...   19656.530000       1.000000

[8 rows x 31 columns]
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [0]:
df=df[['Time','V1','V2','V3','V4','V5','V6','Amount','Class']]

In [0]:
df=df.sample(frac=0.1)
print(df.shape)

(23463, 9)


In [0]:
Fraud = df[df['Class'] == 1]
Valid = df[df['Class'] == 0]
anomaly_fraction = (len(Fraud)/float(len(Valid)))
print('  Anomaly_Fraction : ',anomaly_fraction)
print('       Fraud Cases : {}'.format(len(Fraud)))
print('Valid Transactions : {}'.format(len(Valid)))

  Anomaly_Fraction :  0.001536688436419516
       Fraud Cases : 36
Valid Transactions : 23427


In [0]:
columns = df.columns.tolist()

columns = [c for c in columns if c not in ["Class"]]

target = "Class"

X = df[columns]
Y = df[target]
print(X.shape)
print(Y.shape)

(23463, 8)
(23463,)


In [0]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(
        n_neighbors=20,
        contamination=anomaly_fraction)

In [0]:
plt.figure(figsize=(9, 7))
n_outliers = len(Fraud)
y_pred = clf.fit_predict(X)
scores_pred = clf.negative_outlier_factor_
    # Reshape the prediction values to 0 for valid, 1 for fraud. 
y_pred[y_pred == 1] = 0
y_pred[y_pred == -1] = 1
    
n_errors = (y_pred != Y).sum()
    
    # Run classification metrics
print('Local Outlier Factor: {}'.format( n_errors))
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

Local Outlier Factor: 73
0.9968887184077058
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     23427
         1.0       0.00      0.00      0.00        36

    accuracy                           1.00     23463
   macro avg       0.50      0.50      0.50     23463
weighted avg       1.00      1.00      1.00     23463



<Figure size 648x504 with 0 Axes>

In [0]:
import pickle
with open('Credit_Card_fraud_detection.pkl', 'wb') as file:
    pickle.dump(clf, file)